In [1]:
import pandas as pd

from pandas import DataFrame, Series
import requests
import re
import lxml
from bs4 import BeautifulSoup

import numpy as np

`참고자료:`
- askdjango > pandas
- 구글링

### 크롤링을 하기 위해 필요한 기본 사항

-----------------------------


실제로 데이터를 가져올 웹사이트 주소를 **chart_url** 와 같이 변수에 할당해둔다. 

서버에 Request를 보낼 때 __안정적인 데이터 확보__ 를 위해 User Agent 를 헤더에 담아 `URL주소와 headers` 를 같이 요청한다.

*headers 는 선택사항이지만 크롤링을 하기 위해 꼭 넣어서 서버에 요청하는 것을 권장

*그리고 headers(User Agent) 는 브라우저 중에서 호환성이 관대한 FireFox, Chrome 를 추천

In [2]:
chart_url = 'https://music.bugs.co.kr/chart'

headers = {
    'User_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}

### requests 모듈을 사용하여 Response 200 코드 받기

- `requests` 모듈을 사용하여 chrart_url을 가져오게 되면 HTTP Response 의 응답을 받게 된다. Response 200을 받으면 데이터 파싱이 가능한 상태

```python
> res = requests.get(chart_url, headers)
> res
# output <Response [200]>
```

In [3]:
res = requests.get(chart_url, headers=headers)
html = res.text

## pandas 를 사용하여 본격적으로 크롤링하기

**pandas** 의 자료구조인 `data_frame`과 `series` 로 크롤링 해보기

----------------------

> **DataFrame** 은 2차적 데이터 구조 `2개 이상의 column` 이 존재하면 DataFrame 이라고 생각하자

> **Series** 는 1차적 데이터 구조로 `1개의 column` 을 보여주며, Series를 사용하는 이유는 2개의 데이터베이스를 합칠 경우에 index의 순서가 달라도 스마트하게 동일한 index 끼리의 처리를 할 수 있기 때문이다.


In [4]:
type(pd.read_html(html)) # 데이터베이스 테이블이 list 타입임을 확인

list

In [5]:
len(pd.read_html(html)) # 데이터베이스 테이블(리스트)가 3개임을 확인

3

### 내가 추출하고자 하는 차트의 테이블 값을 추출

In [6]:
chart_df = pd.read_html(html)[0] # 총 3개의 리스트 중에서 첫번째[0] 의 리스트 값을 추출
chart_df

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타
0,NaN,1 0변동없음,NaN,곡정보,내 생에 아름다운,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
1,NaN,2 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
2,NaN,3 0변동없음,NaN,곡정보,Make Up (Feat. Crush),샘김(SAM KIM),Sun And Moon Part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
3,NaN,4 0변동없음,NaN,곡정보,Friend,비투비,Friend,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
4,NaN,5 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
5,NaN,6 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
6,NaN,7 0변동없음,NaN,곡정보,Save (Feat. 팔로알토) (Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
7,NaN,8 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
8,NaN,9 2 계단 상승,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
9,NaN,10 1 계단 하락,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능


### 크롤링한 데이터프레임의 행과 열을 파악하는 방법 ( .shape)

In [7]:
chart_df.shape # 100개의 row 와 13개의 columns 로 이루어져있는 것을 확인

(100, 13)

### columns 를 이용하여 데이터프레임의 열에 속한 필드를 확인 (.columns)

In [8]:
chart_df.columns

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타'],
      dtype='object')

### .set_index 를 이용하여 다시 정렬하기

내가 원하는 레코드의 값만을 지정해서 추출하고 싶을 때가 있는데, `columns=[]` 를 이용하여 추출하고자 하는 필드명을 지정하고, `set_index()`로 기준을 잡아 다시 출력할 수 있다.

-------------------------


In [9]:
pd.DataFrame(chart_df, columns=['순위','곡', '아티스트', '앨범']).set_index('곡')

,순위,아티스트,앨범
곡,,,
내 생에 아름다운,1 0변동없음,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4
삐삐,2 0변동없음,아이유(IU),삐삐
Make Up (Feat. Crush),3 0변동없음,샘김(SAM KIM),Sun And Moon Part.1
Friend,4 0변동없음,비투비,Friend
멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),5 0변동없음,Zion.T,ZZZ
Kiss and Make Up,6 0변동없음,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
Save (Feat. 팔로알토) (Prod. 코드 쿤스트),7 0변동없음,루피(Loopy),쇼미더머니 777 Episode 3
고백,8 0변동없음,양다일,고백
꿈처럼 내린,9 2 계단 상승,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3


### 데이터프레임에 '좋아요' 필드를 추가하고 head, tail 을 사용해보기

- `.head()` 의 기본 추출은 상위 5개의 필드값만 나오며, 10개만 추출하고 싶다면 .head(10)과 같은 옵션을 설정

- `.tail()` 역시 기본 추출은 하위 5개의 필드값이며, 10개만 추출하고 싶다면 .tail(10) 옵션을 설정

데이터프레임에 필드를 추가하는 방식은 파이썬에서 dict 를 추가하는 방식과 비슷하다.

In [10]:
color_dict = {'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [11]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [12]:
color_dict['black'] = '검정색' # dict 타입에 key, value 추가

In [13]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색', 'black': '검정색'}

dict 타입에 key, value 를 추가한 것처럼 DataFrame 에 추가해보자.

In [14]:
chart_df['좋아요'] = 100 # 최초에 좋아요 레코드값을 100으로 통일하여 설정

In [15]:
# 정렬 기준에 따라서 레코드값을 1 ~ 101까지 설정
# 만약 데이터프레임과 설정하려는 레코드값이 다를 경우에는 Length 에러가 발생
## chart_df['좋아요'] = range(1, 30) <-- Length 에러

chart_df['좋아요'] = range(1, 101)

In [16]:
chart_df.head(10)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,내 생에 아름다운,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,Make Up (Feat. Crush),샘김(SAM KIM),Sun And Moon Part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,Friend,비투비,Friend,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5
5,NaN,6 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,6
6,NaN,7 0변동없음,NaN,곡정보,Save (Feat. 팔로알토) (Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,7
7,NaN,8 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,8
8,NaN,9 2 계단 상승,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,9
9,NaN,10 1 계단 하락,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,10


> ### .head() 옵션

**head 옵션을 사용하면 상위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.head() # 기본은 5개

- chart_df.head(50) # 50개의 필드 추출

---------------------------

In [17]:
chart_df.head()

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,내 생에 아름다운,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,Make Up (Feat. Crush),샘김(SAM KIM),Sun And Moon Part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,Friend,비투비,Friend,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5


> ### .tail() 옵션

**tail 옵션을 사용하면 하위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.tail() # 기본은 5개

- chart_df.tail(50) # 50개의 필드 추출

---------------------------

In [18]:
chart_df.tail(7)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
93,NaN,94 7 계단 상승,NaN,곡정보,Roller Coaster,청하,Offset,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,94
94,NaN,95 8 계단 상승,NaN,곡정보,잠꼬대 (Feat. 오혁),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,95
95,NaN,96 1 계단 상승,NaN,곡정보,Thunder,Imagine Dragons(이매진 드래곤스),Evolve,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,96
96,NaN,97 10 계단 상승,NaN,곡정보,You,멜로망스(MeloMance),투유 프로젝트 - 슈가맨2 Part.2,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,97
97,NaN,98 15 계단 하락,NaN,곡정보,FIRE (Prod. GroovyRoom),식케이 (Sik-K),FIRE,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,98
98,NaN,99 3 계단 상승,NaN,곡정보,Sun And Moon,샘김(SAM KIM),Sun And Moon Part.1,듣기,재생목록에 추가,내 앨범에 담기,다운로드,영상 재생 불가,기타 기능,99
99,NaN,100 5 계단 하락,NaN,곡정보,바람의 언덕,홍대광,바람의 언덕,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,100


> ### 레코드값을 이용하여 오름차순, 내림차순하기 `(ascending=True, False)`

-------------------------------

기본옵션은 `True` 로 ascending 옵션을 명시할 필요는 없다.

In [19]:
a = chart_df.sort_values('아티스트', ascending=False) # 아티스트 기준으로 내림차순(황 ~ 특수문자)
a[['곡', '아티스트']]


,곡,아티스트
14,어찌 잊으오,황치열
99,바람의 언덕,홍대광
53,비,폴킴(Paul Kim)
17,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴(Paul Kim)
33,헤어지는 중,펀치 (Punch)
85,Page 0,태연 (TAEYEON) 태연 (TAEYEON)
13,넌 (none),크러쉬(Crush)
0,내 생에 아름다운,케이윌
24,벚꽃연가,첸 (CHEN)
84,Love U,청하


### 좋아요 기준 50개를 넘는 레코드값만 출력하기

----------------------

In [20]:
mask = chart_df['좋아요'] > 50
chart_df[mask]

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
50,NaN,51 1 계단 하락,NaN,곡정보,Forever Young,BLACKPINK,SQUARE UP,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,51
51,NaN,52 2 계단 상승,NaN,곡정보,"셋 셀테니 (1, 2, 3!)",승리,THE GREAT SEUNGRI,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,52
52,NaN,53 11 계단 상승,NaN,곡정보,사랑에 아파본 적 있나요,스탠딩 에그(Standing Egg),사랑에 아파본 적 있나요?,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,53
53,NaN,54 1 계단 상승,NaN,곡정보,비,폴킴(Paul Kim),비,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,54
54,NaN,55 8 계단 하락,NaN,곡정보,그 여름밤,샘김(SAM KIM),Sun And Moon Part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,55
55,NaN,56 4 계단 상승,NaN,곡정보,지나오다,닐로(Nil_O),About You,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,56
56,NaN,57 4 계단 하락,NaN,곡정보,"XXL (Feat. 딥플로우, Dok2)",김효은,쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,57
57,NaN,58 10 계단 하락,NaN,곡정보,SoulMate (Feat. 아이유),지코(Zico),SoulMate,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,58
58,NaN,59 3 계단 상승,NaN,곡정보,그때 헤어지면 돼,로이킴,그때 헤어지면 돼,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,59
59,NaN,60 2 계단 하락,NaN,곡정보,좋은 날,멜로망스(MeloMance),미스터 션샤인 (tvN 주말드라마) OST - Part.5,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,60


## 1차적 데이터 구조를 가진 Series

------------------

**Series 를 사용하는 이유 중에 한가지**

<부산지점>

백팩: 10
캐리어: 34
커피: 101

<서울지점>

캐리어: 3
백팩: 55
커피: 23

`* 2지점의 현재 수량은?`

$ total = 부산지점 + 서울지점

$ output: 백팩: 65, 캐리어: 37, 커피: 124

_이와같이 스마트하게 index 를 대조하여 연산이 가능하다._

In [21]:
obj = pd.Series([3, 5, -1, 2]) # index 는 필수로 지정되며, 직접 지정하지 않을 경우에는 아래와 같이 0, 1, ... 와 같이 자동 지정된다.

obj

0    3
1    5
2   -1
3    2
dtype: int64

In [22]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [23]:
obj = pd.Series([3, 5, -1, 2], index=['순이', '영이', '철수', '군자']) # 필요하다면 직접 인덱스를 지정하자.

obj

순이    3
영이    5
철수   -1
군자    2
dtype: int64

In [24]:
obj.index

Index(['순이', '영이', '철수', '군자'], dtype='object')

In [25]:
obj = pd.Series({'blue': '파란색', 'yellow': '노란색', 'red': '빨간색'}) # dict 타입은 key가 index 가 된다

In [26]:
obj

blue      파란색
yellow    노란색
red       빨간색
dtype: object

### key 를 활용한 value 접근 방법

In [27]:
obj.blue

'파란색'

In [28]:
obj['blue']

'파란색'

In [29]:
obj

blue      파란색
yellow    노란색
red       빨간색
dtype: object

In [30]:
type(chart_df['곡'])

pandas.core.series.Series

In [31]:
chart_df['곡']

0                                 내 생에 아름다운
1                                        삐삐
2                     Make Up (Feat. Crush)
3                                    Friend
4       멋지게 인사하는 법 (Feat. 슬기 of Red Velvet)
5                          Kiss and Make Up
6          Save (Feat. 팔로알토) (Prod. 코드 쿤스트)
7                                        고백
8                                    꿈처럼 내린
9      Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트)
10                                  가을 타나 봐
11                   하루도 그대를 사랑하지 않은 적이 없었다
12                    시간이 들겠지 (Feat. Colde)
13                                 넌 (none)
14                                   어찌 잊으오
15                               사이렌(Siren)
16                            Way Back Home
17    모든 날, 모든 순간 (Every day, Every Moment)
18                       이별길 (GOODBYE ROAD)
19                                   IndiGO
20                                  우리 그만하자
21                                      열애중
22                              

## 크롤링한 데이터 저장하기

------------------------

위에서 크롤링한 데이터를 바로 저장해도 되지만 곡, 아티스트, 앨범만 선택하여 csv, xlsx 파일로 저장해보자

In [32]:
chart_df.columns # 저장할 columns 을 다시 확인해보자

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타', '좋아요'],
      dtype='object')

In [33]:
result_chart = chart_df[['곡', '아티스트', '앨범']] # 곡, 아티스트, 앨범을 DataFrame 으로 result_chart 에 저장

### xlsx 파일, csv 파일로 저장하기

In [34]:
result_chart.to_excel('bugs_top_100.xlsx') # pypi 에서 제공하는 openpyxl 을 설치하고 엑셀 파일로 저장

In [35]:
result_chart.to_csv('bugs_top_100.csv') # csv 파일로 저장

### 저장한 파일을 다시 불러오기

In [36]:
pd.read_csv('bugs_top_100.csv').head() # 저장한 csv 파일을 불러오기

,Unnamed: 0,곡,아티스트,앨범
0,0,내 생에 아름다운,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4
1,1,삐삐,아이유(IU),삐삐
2,2,Make Up (Feat. Crush),샘김(SAM KIM),Sun And Moon Part.1
3,3,Friend,비투비,Friend
4,4,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ


In [37]:
pd.read_excel('bugs_top_100.xlsx').head(10) # xlrd 를 설치 # 엑셀 파일 불러오기

,곡,아티스트,앨범
0,내 생에 아름다운,케이윌,뷰티 인사이드 (JTBC 월화드라마) OST - Part.4
1,삐삐,아이유(IU),삐삐
2,Make Up (Feat. Crush),샘김(SAM KIM),Sun And Moon Part.1
3,Friend,비투비,Friend
4,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ
5,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
6,Save (Feat. 팔로알토) (Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3
7,고백,양다일,고백
8,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3
9,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1


In [38]:
%ls

10minutes_to_pandas.ipynb  bugs.ipynb
Pipfile                    bugs_top_100.csv
Pipfile.lock               bugs_top_100.xlsx
Titanic kaggle.ipynb       kaggle_titanic_train.csv
basic_step.ipynb           weekday.html
